In [1]:
from pyspark.sql import SparkSession, functions as F

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "10g")
    .config("spark.driver.memory", "10g")
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/09/04 10:43:37 WARN Utils: Your hostname, LAPTOP-1A92TAQQ resolves to a loopback address: 127.0.1.1; using 172.18.166.45 instead (on interface eth0)
24/09/04 10:43:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/04 10:43:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/04 10:43:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
import pandas as pd
import geopandas as gpd

# read the shapefile
gdf_postcodes = gpd.read_file('../data/POA_2021_AUST_GDA2020_SHP/POA_2021_AUST_GDA2020.shp')

# drop irrelevant / duplicate columns
gdf_postcodes = gdf_postcodes.drop(['AUS_CODE21', 'AUS_NAME21', 'LOCI_URI21'], axis = 1)
gdf_postcodes = gdf_postcodes.drop(gdf_postcodes.columns[1], axis = 1)

print(f'Number of entries = {len(gdf_postcodes)}')
gdf_postcodes.tail()

Number of entries = 2644


,POA_CODE21,AREASQKM21,SHAPE_Leng,SHAPE_Area,geometry
2639,7469,5893.5125,11.341555,0.636169,"MULTIPOLYGON (((144.79215 -41.46478, 144.79212..."
2640,7470,108.2441,0.781048,0.011725,"POLYGON ((145.57769 -41.7864, 145.57686 -41.78..."
2641,9494,0.0000,0.000000,0.000000,None
2642,9797,0.0000,0.000000,0.000000,None
2643,ZZZZ,0.0000,0.000000,0.000000,None


In [3]:
# drop rows for postcodes outside Australia
indices_to_drop = [2641, 2642, 2643]
existing_indices_to_drop = [i for i in indices_to_drop if i in gdf_postcodes.index]
gdf_postcodes = gdf_postcodes.drop(index = existing_indices_to_drop)

print(f'Number of entries = {len(gdf_postcodes)}')
gdf_postcodes.tail()

Number of entries = 2641


,POA_CODE21,AREASQKM21,SHAPE_Leng,SHAPE_Area,geometry
2636,7466,68.8166,0.737044,0.007487,"POLYGON ((145.58776 -42.0603, 145.5875 -42.059..."
2637,7467,144.0971,0.765625,0.015677,"POLYGON ((145.58258 -42.00061, 145.58266 -42.0..."
2638,7468,309.9494,2.114872,0.033768,"MULTIPOLYGON (((145.39701 -42.13911, 145.39774..."
2639,7469,5893.5125,11.341555,0.636169,"MULTIPOLYGON (((144.79215 -41.46478, 144.79212..."
2640,7470,108.2441,0.781048,0.011725,"POLYGON ((145.57769 -41.7864, 145.57686 -41.78..."


In [4]:
merged_transactions_sa2_df = spark.read.parquet('../data/merged/merged_transactions_sa2.parquet')
print(f'Number of entries = {merged_transactions_sa2_df.count()}')
merged_transactions_sa2_df.limit(5)

Number of entries = 14195717


user_id,merchant_abn,dollar_value,order_id,order_datetime,merchant_name,tags,consumer_id,name,address,state,postcode,gender,consumer_fraud_order_datetime,consumer_fraud_probability,merchant_fraud_order_datetime,merchant_fraud_probability,SA2_CODES
1,98973094975,86.97955945703498,2560f7b0-ee5d-4b3...,2022-08-20,Ornare Fusce Inc.,"[(hobby, toy and ...",1195503,Yolanda Williams,413 Haney Gardens...,WA,6935,Female,NULL,NULL,NULL,NULL,NULL
12963,65629452396,10.904622187867213,8e33384c-1047-490...,2021-12-29,Interdum Curabitu...,"((watch, clock, a...",228783,David Flores,7515 Alexander Lane,NSW,1005,Male,NULL,NULL,NULL,NULL,NULL
1,89502033586,124.18468694868491,f8891626-f098-45b...,2022-08-20,Adipiscing Enim LLC,([digital goods: ...,1195503,Yolanda Williams,413 Haney Gardens...,WA,6935,Female,NULL,NULL,NULL,NULL,NULL
12965,47721432444,96.3318301999319,29926d96-2561-4e7...,2021-12-29,Ultrices Posuere ...,"[[gift, card, nov...",269428,Rebecca Taylor,684 Beth Ridges,NSW,2520,Female,NULL,NULL,NULL,NULL,NULL
11144,66079287213,28.93202619064808,8d3549df-cb5a-4f7...,2022-08-20,Hendrerit Corpora...,"((gift, card, nov...",1354980,Matthew Quinn,69987 Lucas Ville,SA,5810,Male,NULL,NULL,NULL,NULL,NULL


In [5]:
postcode_df = merged_transactions_sa2_df.select("postcode", "state")
postcode_df

postcode,state
6935,WA
1005,NSW
6935,WA
2520,NSW
5810,SA
2520,NSW
5810,SA
1741,NSW
5810,SA
1741,NSW


In [7]:
from pyspark.sql import functions as F

# convert geometry to wkt for conversion to Spark
gdf_postcodes['geometry_wkt'] = gdf_postcodes['geometry'].apply(lambda geom: geom.wkt)
gdf_no_geom = gdf_postcodes.drop(columns = 'geometry')

# convert to spark df
spark_gdf_postcodes = spark.createDataFrame(gdf_no_geom)

# join on postcode
transactions_sa2_geom_df = merged_transactions_sa2_df.join(F.broadcast(spark_gdf_postcodes), 
                                                           merged_transactions_sa2_df['postcode'] == spark_gdf_postcodes['POA_CODE21'],
                                                           how = 'Left')
transactions_sa2_geom_df = transactions_sa2_geom_df.drop('POA_CODE21')

print(f'Number of entries = {transactions_sa2_geom_df.count()}')
transactions_sa2_geom_df.limit(5)

24/09/04 10:37:45 WARN TaskSetManager: Stage 10 contains a task of very large size (8928 KiB). The maximum recommended task size is 1000 KiB.


Number of entries = 14195717


24/09/04 10:37:48 WARN TaskSetManager: Stage 14 contains a task of very large size (8928 KiB). The maximum recommended task size is 1000 KiB.
24/09/04 10:37:51 WARN TaskSetManager: Stage 16 contains a task of very large size (8928 KiB). The maximum recommended task size is 1000 KiB.


user_id,merchant_abn,dollar_value,order_id,order_datetime,merchant_name,tags,consumer_id,name,address,state,postcode,gender,consumer_fraud_order_datetime,consumer_fraud_probability,merchant_fraud_order_datetime,merchant_fraud_probability,SA2_CODES,AREASQKM21,SHAPE_Leng,SHAPE_Area,geometry_wkt
1,98973094975,86.97955945703498,2560f7b0-ee5d-4b3...,2022-08-20,Ornare Fusce Inc.,"[(hobby, toy and ...",1195503,Yolanda Williams,413 Haney Gardens...,WA,6935,Female,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
12963,65629452396,10.904622187867213,8e33384c-1047-490...,2021-12-29,Interdum Curabitu...,"((watch, clock, a...",228783,David Flores,7515 Alexander Lane,NSW,1005,Male,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1,89502033586,124.18468694868491,f8891626-f098-45b...,2022-08-20,Adipiscing Enim LLC,([digital goods: ...,1195503,Yolanda Williams,413 Haney Gardens...,WA,6935,Female,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
12965,47721432444,96.3318301999319,29926d96-2561-4e7...,2021-12-29,Ultrices Posuere ...,"[[gift, card, nov...",269428,Rebecca Taylor,684 Beth Ridges,NSW,2520,Female,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
11144,66079287213,28.93202619064808,8d3549df-cb5a-4f7...,2022-08-20,Hendrerit Corpora...,"((gift, card, nov...",1354980,Matthew Quinn,69987 Lucas Ville,SA,5810,Male,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [6]:

# convert geometry to wkt for conversion to Spark
gdf_postcodes['geometry_wkt'] = gdf_postcodes['geometry'].apply(lambda geom: geom.wkt)
gdf_no_geom = gdf_postcodes.drop(columns = 'geometry')

# convert to spark df
spark_gdf_postcodes = spark.createDataFrame(gdf_no_geom)

# join on postcode
postcode_df = postcode_df.join(F.broadcast(spark_gdf_postcodes), 
                                                           postcode_df['postcode'] == spark_gdf_postcodes['POA_CODE21'],
                                                           how = 'Left')
postcode_df = postcode_df.drop('POA_CODE21')

print(f'Number of entries = {postcode_df.count()}')
postcode_df.limit(5)

24/09/04 10:43:54 WARN TaskSetManager: Stage 8 contains a task of very large size (8928 KiB). The maximum recommended task size is 1000 KiB.


Number of entries = 14195717


24/09/04 10:43:57 WARN TaskSetManager: Stage 12 contains a task of very large size (8928 KiB). The maximum recommended task size is 1000 KiB.
24/09/04 10:43:59 WARN TaskSetManager: Stage 14 contains a task of very large size (8928 KiB). The maximum recommended task size is 1000 KiB.


postcode,state,AREASQKM21,SHAPE_Leng,SHAPE_Area,geometry_wkt
6935,WA,NULL,NULL,NULL,NULL
1005,NSW,NULL,NULL,NULL,NULL
6935,WA,NULL,NULL,NULL,NULL
2520,NSW,NULL,NULL,NULL,NULL
5810,SA,NULL,NULL,NULL,NULL


In [7]:
from shapely import wkt
import geopandas as gpd
import pandas as pd

# take 100,000 sample from transactions_sa2_geom
sample_sdf = transactions_sa2_geom_df.sample(withReplacement = False, fraction = 5000 / transactions_sa2_geom_df.count(), seed = 13)

# convert sample to pandas
sample_pdf = sample_sdf.toPandas()

# convert the WKT geometry back to geometry
sample_pdf['geometry'] = sample_pdf['geometry_wkt'].apply(wkt.loads)

# convert Pandas df to Geo df
sample_gdf = gpd.GeoDataFrame(sample_pdf, geometry = 'geometry')
sample_gdf = sample_gdf.drop(columns = 'geometry_wkt')

print(f'Number of entries = {len(sample_gdf)}')
sample_gdf.head()

NameError: name 'transactions_sa2_geom_df' is not defined

In [8]:
from shapely import wkt
import geopandas as gpd
import pandas as pd

# take 100,000 sample from transactions_sa2_geom
sample_sdf = postcode_df.sample(withReplacement = False, fraction = 5000 / postcode_df.count(), seed = 13)

# convert sample to pandas
sample_pdf = sample_sdf.toPandas()

# convert the WKT geometry back to geometry
sample_pdf['geometry'] = sample_pdf['geometry_wkt'].apply(wkt.loads)

# convert Pandas df to Geo df
sample_gdf = gpd.GeoDataFrame(sample_pdf, geometry = 'geometry')
sample_gdf = sample_gdf.drop(columns = 'geometry_wkt')

print(f'Number of entries = {len(sample_gdf)}')
sample_gdf.head()

24/09/04 10:44:08 WARN TaskSetManager: Stage 16 contains a task of very large size (8928 KiB). The maximum recommended task size is 1000 KiB.
24/09/04 10:44:09 WARN TaskSetManager: Stage 20 contains a task of very large size (8928 KiB). The maximum recommended task size is 1000 KiB.


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 33128)
Traceback (most recent call last):
  File "/usr/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/home/chengmingliew/.local/lib/python3.10/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/home/chengmingliew/.local/lib/python3.10/site-packages/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
  File "/home/chengmingliew/.local/lib/python3.10/site-packages/pyspark/accumulators.py", line 2

Number of entries = 4869


,postcode,state,AREASQKM21,SHAPE_Leng,SHAPE_Area,geometry
0,1162,NSW,NaN,NaN,NaN,None
1,1805,NSW,NaN,NaN,NaN,None
2,1314,NSW,NaN,NaN,NaN,None
3,9002,QLD,NaN,NaN,NaN,None
4,6761,WA,NaN,NaN,NaN,None


In [9]:
# count transactions per postcode
postcode_stats = sample_gdf.groupby('postcode').agg(
                                                    num_transactions = ('postcode', 'count'),
                                                    # Keep other stats as well
                                                    state = ('state', 'first'),
                                                    geometry = ('geometry', 'first')
                                                    ).reset_index()

postcode_stats = gpd.GeoDataFrame(postcode_stats, geometry = 'geometry')

# Sort and display
postcode_stats = postcode_stats.sort_values(by = 'num_transactions', ascending = False)
postcode_stats.head(20)

,postcode,num_transactions,state,geometry
536,2519,8,NSW,"POLYGON ((150.88228 -34.38119, 150.88323 -34.3..."
1954,6078,8,WA,"POLYGON ((115.81735 -31.54366, 115.8552 -31.54..."
661,2755,7,NSW,None
1958,6083,7,WA,"POLYGON ((116.19674 -31.82202, 116.19665 -31.8..."
941,3322,7,VIC,"POLYGON ((143.65584 -37.98136, 143.66955 -37.9..."
395,2232,7,NSW,"POLYGON ((151.07641 -34.04783, 151.07642 -34.0..."
1289,4002,7,QLD,None
1903,5950,6,SA,"POLYGON ((138.5475 -34.93819, 138.54757 -34.93..."
78,1130,6,NSW,None
2310,7109,6,TAS,"MULTIPOLYGON (((146.88218 -43.58616, 146.88261..."


In [8]:
# plot number of transactions per postcode in Victoria
# group into smaller df for Victoria

postcode_stats_victoria = postcode_stats[postcode_stats['state'] == 'VIC']
postcode_stats_victoria.head()

,postcode,num_transactions,state,geometry
941,3322,7,VIC,"POLYGON ((143.65584 -37.98136, 143.66955 -37.9..."
1157,3747,6,VIC,"POLYGON ((146.80991 -36.44739, 146.80353 -36.4..."
1111,3664,6,VIC,"POLYGON ((145.43169 -36.9001, 145.43137 -36.90..."
1174,3779,6,VIC,"POLYGON ((145.73752 -37.48189, 145.74735 -37.4..."
868,3194,5,VIC,"POLYGON ((145.09133 -37.98441, 145.09075 -37.9..."


In [ ]:
import folium
from folium import Choropleth

# From MAST30034 - Tute2_python.ipynb (Liam Hodgkinson and Mingming Gong)
# Determine starting position of map
m = folium.Map(location = [-25, 134], tiles = "Stamen Terrain",
               attr="<a href = https://github.com/liamhodg/MAST30034_Python/tree/main/tutorials>MAST30034</a>",
               zoom_start = 5)

# Convert passenger_stats to GeoJSON format
gdf_json = postcode_stats.to_json()

# Create map
Choropleth(
    geo_data = gdf_json,
    name = 'Transactions by Postcode (in Victoria)',
    data = postcode_stats,
    columns = ['postcode', 'num_transactions'],
    key_on = 'feature.properties.postcode',
    fill_color = 'YlGnBu',
    fill_opacity = 1,
    line_opacity = 0.4,
    legend_name = 'Number of Transactions'
).add_to(m)

folium.LayerControl().add_to(m)
# m.save('../plots/geospatial/n_transactions_per_postcode_vic.html')
m

In [13]:
m.save('../plots/geospatial/n_transactions_per_postcode.html')